In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from fugu_modules import Model
def setup_plot(y_limits=None):
    """Set up the plot with common settings."""
    fig = plt.figure(figsize=(10, 5)) 
    ax = plt.gca()
    fig.fontsize = 12
    plt.tight_layout()
    ax.grid(True)
    # ax.xaxis.set_major_locator(MultipleLocator(2))
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x)}'))
    
    if y_limits is not None:
        ax.set_ylim(y_limits)
    
    return ax
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

In [2]:
# Training
from sklearn.model_selection import train_test_split
df = pd.read_csv( '../netrep_100ms.csv') # from net replica
df2 = pd.read_csv('../oct_100ms.csv') # from puffer
# add df2 to df

# TODO: can add df and df2 together to get a combined dataset (may need to shuffle)
# df = pd.concat([df, df2])

df['in'] = df.iloc[:, :62].apply(list, axis=1) # 62 input features
input_data = np.array(df['in'].tolist())
output_data = np.array(df['actual'].tolist()) # 1 output feature

model = Model()
train_input, test_input, train_output, test_output = train_test_split(
    input_data, output_data, test_size=0.05, random_state=42
)

model.train(train_input, train_output, test_input, test_output,
            model_path='../models/combine_puffer_netrep.pt')
 

Epoch 1/10, Loss: 1.3823356551483537
Length of predictions:  32577
Length of actual:  32577
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
Test Mean Squared Error: 49.11817242609817
Epoch 2/10, Loss: 1.2024289118792606
Length of predictions:  32577
Length of actual:  32577
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
Test Mean Squared Error: 48.73335050419007
Epoch 3/10, Loss: 1.156477730247339
Length of predictions:  32577
Length of actual:  32577
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
Test Mean Squared Error: 48.38678218144703
Epoch 4/10, Loss: 1.1282767129104372
Length of 

In [3]:
# evaluate test set with trustee
model.evaluate_with_trustee(test_input, test_output, "evals/")

Test Cross-Entropy Loss: 2.5374941576143075
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       642
           1       0.71      0.44      0.54      4681
           2       0.42      0.37      0.40      6849
           3       0.43      0.54      0.48      7285
           4       0.31      0.55      0.40      3362
           5       0.34      0.31      0.32      1676
           6       0.26      0.01      0.01       698
           7       0.18      0.02      0.03       384
           8       0.00      0.00      0.00       287
           9       0.02      0.00      0.01       302
          10       0.06      0.01      0.02       433
          11       0.03      0.01      0.01       562
          12       0.03      0.01      0.01       765
          13       0.01      0.00      0.00       675
          14       0.00      0.00      0.00       517
          15       0.00      0.00      0.00       453
          16  

/Users/coltersirlin/.pyenv/versions/cs293/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/coltersirlin/.pyenv/versions/cs293/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/coltersirlin/.pyenv/versions/cs293/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/coltersirlin/.pyenv/versions/cs293/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: Und

## Evaluate based on latency 

In [4]:
test = pd.read_csv('/Users/coltersirlin/Desktop/cs293n/netrep_100ms.csv')
print(test['trans_time0'].describe())
test['in'] = test.iloc[:, :62].apply(list, axis=1)

test_input = np.array(test['in'].tolist())
test_output = np.array(test['actual'].tolist())

count    651528.000000
mean          3.976824
std           4.759738
min           0.120000
25%           0.995000
50%           1.775000
75%           6.312000
max         162.436000
Name: trans_time0, dtype: float64


In [5]:
model = Model()
# you need to download this model from puffer website or use the one I put here
model.load('/Users/coltersirlin/Desktop/cs293n/bbr-20221001-1/py-4-checkpoint-200.pt')
model.evaluate(test_input, test_output, '../evals/OctAll_s_eval.csv')


/Users/coltersirlin/Desktop/cs293n/cs293-underspec/fugu_modules.py:331: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)


Length of predictions:  651528
Length of actual:  651528
1000
2000
3000


KeyboardInterrupt: 

# Evaluate the model

In [ ]:
test_df = pd.read_csv('/Users/coltersirlin/Desktop/cs293n/netrep_100ms.csv')
# test_df = pd.read_csv('test_allFields_ttpABR14_min100rtt_max4Mdelivery.csv')
# trans_time_cols = [col for col in test_df.columns if col.startswith('trans_time')]
# test_df[trans_time_cols] = test_df[trans_time_cols] / 1000


print(test_df['trans_time0'].describe())
test_df['in'] = test_df.iloc[:, :62].apply(list, axis=1)

test_input = np.array(test_df['in'].tolist())
test_output = np.array(test_df['actual'].tolist())

In [ ]:
model = Model()
model.load('/mnt/md0/jaber/puffer_trustee/netrep_fineTune/model_6mCap.pt')
# model.load('/mnt/md0/jaber/puffer_trustee/netrep_model_4m_100ms/corrected_model.pt')
# model.load('/mnt/md0/vamsi995/puffer_retrain/bbr-20221001-1/py-4-checkpoint-200.pt')
# model.load('/mnt/md0/jaber/puffer_trustee/feb_bbr-20190202-1/py-4-checkpoint-100.pt')
# model.load('/mnt/md0/jaber/puffer_trustee/bbr-20221001-1/py-4-checkpoint-200.pt')
# model.load('/mnt/md0/jaber/puffer_trustee/puffer_fineTune/model_6mCap.pt')
# model.load('/mnt/md0/jaber/puffer_trustee/puffer_fineTune/combine_puffer_netrep.pt')
model.evaluate(test_input, test_output, 'fineTune_onNetRep.csv')


In [ ]:
netrep = pd.read_csv('fineTune_onNetRep.csv')
fugu = pd.read_csv('fugu22onNetRep.csv')

In [ ]:
print("mse piont:" , mean_squared_error(result["actual"], result["point_predict"]))
print("mse prob", mean_squared_error(result["actual"], result["prob_predict"]))

print("MAPE piont:" , mean_absolute_percentage_error(result["actual"], result["point_predict"]))
print("MAPE prob", mean_absolute_percentage_error(result["actual"], result["prob_predict"]))

In [ ]:


# Plot CDF
plt.figure(figsize=(8, 5))
for col, label in zip(["error_point", "error_prob"], ["Point Estimate", "Probabilistic"]):
    sorted_errors = np.sort(result[col])
    cdf = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
    plt.plot(sorted_errors, cdf, label=label)

plt.xlabel("Absolute Prediction Error")
plt.ylabel("CDF")
plt.legend()
plt.xscale('log')
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
for col, label in zip(["error_point", "error_prob"], ["Point Estimate", "Probabilistic"]):
    sorted_errors = np.sort(result[col])
    cdf = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
    plt.plot(sorted_errors, cdf, label=label)

plt.xlabel("Prediction Error")
plt.ylabel("CDF")
plt.legend()
plt.grid()
plt.show()

In [ ]:
df = pd.read_csv('netrep_on_netrep_AlexFunc.csv')
df1 = pd.read_csv('netrep_on_netrep.csv')

In [ ]:
# # read pkl file 
# import pickle
# with open('/mnt/md0/jaber/puffer_trustee/onAndOff_4m_pfifo_latency100_4m/onAndOff_latency.pkl', 'rb') as f:
#     df1 = pickle.load(f)

In [ ]:
df = pd.read_csv('/mnt/md0/jaber/puffer_trustee/ttpABR_14days/train_netReplica_min100rtt_max6Mdelivery.csv')

In [ ]:
df['trans_time0'].describe()

# Training

In [ ]:
### with the CSV data or PKL

In [ ]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('../netrep_100ms.csv')
df['in'] = df.iloc[:, :62].apply(list, axis=1)
df['trans_time2'].describe()

# Convert the trans_time columns if the scale is not right, it should be in seconds not ms 
# trans_time_cols = [col for col in test_df.columns if col.startswith('trans_time')]
# test_df[trans_time_cols] = test_df[trans_time_cols] / 1000

input_data = np.array(df['in'].tolist())
# should we use predict as train output or the actual output?
output_data = np.array(df['actual'].tolist())
# IMPORTANT: Change the transtime scale for input too 


model = Model()
train_input, test_input, train_output, test_output = train_test_split(
    input_data, output_data, test_size=0.01, random_state=42)
model.train(train_input, train_output, test_input, test_output,
            model_path='../netrep_100ms.pt')



In [ ]:

df = pd.read_csv('/mnt/md0/vamsi995/puffer_retrain/data/train_netReplica_min100rtt_max4Mdelivery.csv')

df['in'] = df.iloc[:, :62].apply(list, axis=1)

train_input = np.array(df['in'].tolist())
# should we use predict as train output or the actual output?
train_output = np.array(df['actual'].tolist())


# Fine tune

In [ ]:
## Find Tuning 
from sklearn.model_selection import train_test_split
df = pd.read_csv( '../netrep_100ms.csv')
df2 = pd.read_csv('../oct_100ms.csv')
# add df2 to df

# TODO: uncomment
# df = pd.concat([df, df2])

print(len(df))

# trans_time_cols = [col for col in df.columns if col.startswith('trans_time')]
# df[trans_time_cols] = df[trans_time_cols] / 1000
print(df['trans_time0'].describe())

df['in'] = df.iloc[:, :62].apply(list, axis=1)

input_data = np.array(df['in'].tolist())
# should we use predict as train output or the actual output?
output_data = np.array(df['actual'].tolist())

# model = Model()
# model.load('/mnt/md0/vamsi995/puffer_retrain/train_models/fine_tune.pt')


# print(len(processed_data[0]['in']))
# print(len(processed_data[0]['out']))


# model.load_model("/mnt/md0/vamsi995/puffer_retrain/bbr-20221001-1/cpp-4-checkpoint-200.pt", "/mnt/md0/vamsi995/puffer_retrain/bbr-20221001-1/cpp-meta-4-checkpoint-200.json")
# model = Model()
# model.load("/mnt/md0/vamsi995/puffer_retrain/train_models/fine_tune_all_data.pt")


model = Model()
train_input, test_input, train_output, test_output = train_test_split(
    input_data, output_data, test_size=0.01, random_state=42
)
model.train(train_input, train_output, test_input, test_output,
            model_path='../models/combine_puffer_netrep.pt')


# Model performance visual

In [ ]:
df1 = pd.read_csv('netrep_on_netrep.csv')
df2 = pd.read_csv('netrep_on_puffer.csv')
df3 = pd.read_csv('puffer_on_puffer.csv')
df4 = pd.read_csv('puffer_on_netrep.csv')
df5 = pd.read_csv('fugu_on_netrep.csv')
df6 = pd.read_csv('fugu_on_puffer.csv')
df7 = pd.read_csv('fuguFeb_on_netrep.csv')
df8 = pd.read_csv('fuguFeb_on_puffer.csv')
df9 = pd.read_csv('fineTune_netrep_on_netrep.csv')
df10 = pd.read_csv('fineTune_netrep_on_puffer.csv')

# column names are trans_time0 to trans_time7 and then predict and actual time
# columns = ['trasn_time0', 'trans_time1', 'trasn_time2','trasn_time3','trasn_time4','trasn_time5','trasn_time6', 'trasn_time7', 'predict', 'actual']
# df1.columns = columns
# df2.columns = columns
# df3.columns = columns
# df4.columns = columns
# df5.columns = columns

# # claculate the error which is the difference between the actual and the predicted time
# df1['error'] = abs(df1['actual'] - df1['predict']) * 1000
# df2['error'] = abs(df2['actual'] - df2['predict']) * 1000
# df3['error'] = abs(df3['actual'] - df3['predict']) * 1000
# df4['error'] = abs(df4['actual'] - df4['predict']) * 1000
# df5['error'] = abs(df5['actual'] - df5['predict']) * 1000



In [ ]:
df1['point_error'] = df1['point_error'] * 1000
df2['point_error'] = df2['point_error'] * 1000
df3['point_error'] = df3['point_error'] * 1000
df4['point_error'] = df4['point_error'] * 1000
df5['point_error'] = df5['point_error'] * 1000
df6['point_error'] = df6['point_error'] * 1000
df7['point_error'] = df7['point_error'] * 1000
df8['point_error'] = df8['point_error'] * 1000
df9['point_error'] = df9['point_error'] * 1000
df10['point_error'] = df10['point_error'] * 1000


In [ ]:
dfs = {
    "Model: netReplica,  Test: netReplica": df1,
    # "Model: netReplica,  Test: Puffer": df2,
    "Model: Puffer,  Test: netReplica": df4,
    "Model: Puffer, Test: Puffer": df3, 
    "Model: Fugu,  Test: netReplica": df5,
    # "Model: Fugu,  Test: Puffer": df6,
    "Model: FuguFeb,  Test: netReplica": df7,
    # "Model: FuguFeb,  Test: Puffer": df8,
    "Model: Fine Tuned with netReplica,  Test: netReplica": df9
    # "Model: Fine Tuned with netReplica,  Test: Puffer": df10
}

In [ ]:
# print the 96th percentile of the error for all of dhet modesl
for key, df in dfs.items():
    print(key, df['point_error'].quantile(0.96))

In [ ]:
# plot the cdf for the error greter thatn 95th percentile
fig = plt.figure(figsize=(10, 5))
ax = plt.gca()
fig.fontsize = 12
plt.tight_layout()
ax.grid(True)
ax.set_xscale('log')
ax.set_xlabel('Error (ms)')
ax.set_ylabel('CDF')
for key, df in dfs.items():
    df['point_error'].hist(cumulative=True, bins=1000, density=True, histtype='step', label=key)
plt.legend()
plt.show()

In [ ]:
# plot the cdf of error on df1 using line plot
ax = setup_plot()
for label, df in dfs.items():
    sorted_errors = np.sort(df["prob_error"])
    cdf = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
    plt.plot(sorted_errors, cdf, label=label)

plt.xlabel("Prediction Error (ms)")
plt.ylabel("CDF")
plt.title("CDF of TTP Probabilistic Error for Different Models Tested on Different Datasets")
plt.legend()
plt.xscale('log')
plt.xlim(1 , 12000)
plt.grid(True)
plt.show()

# Fugu vs Netrep points
### Find the points were fugu was doing well not the netRplica and vice versa

In [ ]:
netrep = pd.read_csv('fineTune_netrep_on_netrep.csv')
fugu = pd.read_csv('fugu_on_netrep.csv')
test = pd.read_csv('/mnt/md0/vamsi995/puffer_retrain/data/test_netReplica_min100rtt_max4Mdelivery.csv')

In [ ]:
diff = netrep["point_error"] - fugu["point_error"]

In [ ]:
# netrep better than netrep 
data = test.iloc[diff.nlargest(7).index]

In [ ]:
# append the point_error of netrep and fugu to the data df
data['netrep_point_error'] = netrep.iloc[diff.nlargest(7).index]['point_error'].values
data['fugu_point_error'] = fugu.iloc[diff.nlargest(7).index]['point_error'].values

In [ ]:
data[trans_time_cols]

In [ ]:
# Dropping for netReplica outperforms fugu
# data = data.drop(data.index[0])
# data = data.drop(data.index[0])

# Dropping for fugu outperforms netReplica
data = data.drop(data.index[0])
data = data.drop(data.index[2])


### Samples with high and low error 

In [ ]:
trans_time_cols = [col for col in data.columns if col.startswith('trans_time')]
rtt_cols = [col for col in data.columns if col.startswith('rtt')]
delivery_cols = [col for col in data.columns if col.startswith('delivery_rate')]
data[delivery_cols] = data[delivery_cols] * (8 * 1500)  / 1000000
data[rtt_cols] = data[rtt_cols] * 1000
data[trans_time_cols] = data[trans_time_cols] / 1000

In [ ]:
import matplotlib.pyplot as plt

# Define a function to set up the plot
# def setup_plot():
#     fig, ax = plt.subplots(figsize=(10, 5))
#     ax.set_title("Time Series Data (Chunks with Large Sigma)")
#     ax.set_xlabel("Index")
#     ax.set_ylabel("Values")
#     return ax

# Function to plot time series
def plot_time_series(columns_prefix, ax, label, index_range):
    for row_index in range(data.shape[0]):  # Iterate over rows in the DataFrame
        values = [data[f"{columns_prefix}{i}"].iloc[row_index] for i in index_range]
        # add prediction error and sigma to the plot label
        label = f"Fugu Error: {data['fugu_point_error'].iloc[row_index]:.2f}s, NetRep Error: {data['netrep_point_error'].iloc[row_index]:.2f}s"
        ax.plot(index_range, values, label=f"{row_index + 1}: {label} ")

# List of metrics and their respective index ranges
metrics = {
    "rtt": range(9),
    "trans_time": range(8),
    "cwnd": range(9),
    "delivery_rate": range(9),
    "size": range(9),
}

# Loop through each metric and generate the plot
# for metric, index_range in metrics.items():
#     ax = setup_plot()
#     plot_time_series(metric, ax, metric.capitalize(), index_range)
#     ax.legend()
#     plt.show()
    
ax = setup_plot()
plot_time_series('rtt', ax, "RTT (ms)", range(9))
ax.set_title("RTT of Past 8 Chunks (Fugo Outperforms Fine Tuned netReplica Model)")
ax.set_ylabel("RTT (ms)")
ax.set_xlabel("Chunk Number")
ax.legend(loc='upper left', fontsize='small')
plt.show()

ax = setup_plot()
plot_time_series('trans_time', ax, "Transmission Time (s)", range(8))
ax.set_title("Transmission Time of Past 8 Chunks (Fugo Outperforms Fine Tuned netReplica Model)")
ax.set_ylabel("Transmission Time (s)")
ax.set_xlabel("Chunk Number")
ax.legend(loc='upper left', fontsize='small')
plt.show()

ax = setup_plot()
plot_time_series('delivery_rate', ax, "Delivery Rate (Mbps)", range(9))
ax.set_title("Delivery Rate of Past 8 Chunks (Fugo Outperforms Fine Tuned netReplica Model)")
ax.set_ylabel("Delivery Rate (Mbps)")
ax.set_xlabel("Chunk Number")
ax.legend(loc='upper left', fontsize='small')
plt.show()

ax = setup_plot()
plot_time_series('size', ax, "Size (Bytes)", range(9))
ax.set_title("Size of Past 8 Chunks (Fugo Outperforms Fine Tuned netReplica Model)") 
ax.set_ylabel("Size (Bytes)")
ax.set_xlabel("Chunk Number")
# position the legent on top left
ax.legend(loc='upper left', fontsize='small')
plt.show()